# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Rabaul,-4.1967,152.1721,300.68,76,58,1.09,PG,1714253995
1,1,Edinburgh of the seven seas,-37.0676,-12.3116,285.59,71,98,12.48,SH,1714253899
2,2,Kaiapoi,-43.3783,172.6401,289.57,77,9,1.03,NZ,1714253995
3,3,Bethel,41.3712,-73.4140,288.26,37,0,6.17,US,1714253807
4,4,Jamestown,42.0970,-79.2353,291.88,71,99,5.81,US,1714253904


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.01,
    color = "City"
)
# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_mod0 = city_data_df.loc[(city_data_df["Humidity"]<50)]
city_data_mod = city_data_mod0.loc[(city_data_mod0["Max Temp"]<299)]
city_data_ideal = city_data_mod.loc[(city_data_mod["Cloudiness"]==0)]
# Drop any rows with null values
city_data_clean = city_data_ideal.dropna()

# Display sample data
city_data_clean

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,Bethel,41.3712,-73.4140,288.26,37,0,6.17,US,1714253807
41,41,Mount isa,-20.7333,139.5000,290.02,48,0,2.57,AU,1714253998
43,43,Cole harbour,44.6724,-63.4751,283.25,34,0,2.68,CA,1714253941
87,87,Zhosaly,45.4878,64.0781,285.42,20,0,4.22,KZ,1714254000
100,100,Badger,64.8000,-147.5333,285.29,38,0,3.60,US,1714253921
110,110,Thompson,55.7435,-97.8558,273.24,34,0,2.57,CA,1714253844
202,202,Rikaze,29.2500,88.8833,277.07,40,0,2.31,CN,1714254009
282,282,Al kharijah,25.4514,30.5464,294.71,17,0,4.11,EG,1714253931
302,302,Watertown,42.3709,-71.1828,292.42,39,0,5.36,US,1714253844
380,380,Laguna,38.4210,-121.4238,297.69,40,0,3.60,US,1714253915


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_clean[["City","Country","Humidity","Lat","Lng"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Humidity,Lat,Lng,Hotel Name
3,Bethel,US,37,41.3712,-73.4140,
41,Mount isa,AU,48,-20.7333,139.5000,
43,Cole harbour,CA,34,44.6724,-63.4751,
87,Zhosaly,KZ,20,45.4878,64.0781,
100,Badger,US,38,64.8000,-147.5333,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
categories = "accommodation.hotel"
params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    print(name_address.url)
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=b54720ef51214bb2ad188213d1e0ea51&filter=circle%3A-73.414%2C41.3712%2C10000&bias=proximity%3A-73.414%2C41.3712
Bethel - nearest hotel: Hampton Inn Danbury
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=b54720ef51214bb2ad188213d1e0ea51&filter=circle%3A139.5%2C-20.7333%2C10000&bias=proximity%3A139.5%2C-20.7333
Mount isa - nearest hotel: Ibis Styles
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=b54720ef51214bb2ad188213d1e0ea51&filter=circle%3A-63.4751%2C44.6724%2C10000&bias=proximity%3A-63.4751%2C44.6724
Cole harbour - nearest hotel: Hearthstone Inn Boutique Hotel Dartmouth/Halifax
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=b54720ef51214bb2ad188213d1e0ea51&filter=circle%3A64.0781%2C45.4878%2C10000&bias=proximity%3A64.0781%2C45.4878
Zhosaly - nearest hotel: Gostinitsa Yeseta
https://a

,City,Country,Humidity,Lat,Lng,Hotel Name
3,Bethel,US,37,41.3712,-73.4140,Hampton Inn Danbury
41,Mount isa,AU,48,-20.7333,139.5000,Ibis Styles
43,Cole harbour,CA,34,44.6724,-63.4751,Hearthstone Inn Boutique Hotel Dartmouth/Halifax
87,Zhosaly,KZ,20,45.4878,64.0781,Gostinitsa Yeseta
100,Badger,US,38,64.8000,-147.5333,Clarion Hotel & Suites
110,Thompson,CA,34,55.7435,-97.8558,Thompson Inn
202,Rikaze,CN,40,29.2500,88.8833,Gesar Hotel
282,Al kharijah,EG,17,25.4514,30.5464,Kharga Hotel
302,Watertown,US,39,42.3709,-71.1828,Four Points by Sheraton
380,Laguna,US,40,38.4210,-121.4238,Holiday Inn Express & Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600, 
    scale = 0.5,
    color = "City",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)